### S&P500 VIX Analysis

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import utils

load_dotenv()
file_path = os.getenv("FILE_PATH")
SNP_VIX_file = f'{file_path}/S&P 500 VIX.csv'

In [2]:
# Load the CSV file into df
VIX = pd.read_csv(SNP_VIX_file)
VIX.columns = ["vol"]
VIX["log_vol_diff"] = np.log(VIX['vol']/ VIX['vol'].shift(1))
VIX.index = pd.to_datetime(VIX.index)
VIX = VIX.sort_index(ascending=True)
VIX = VIX[1:]
VIX['log_vol'] = np.log(VIX['vol'])

VIX.head()

,vol,log_vol_diff,log_vol
2015-01-05,19.92,-0.058496,2.991724
2015-01-06,21.12,0.089597,3.050220
2015-01-07,19.31,0.126822,2.960623
2015-01-08,17.01,-0.031253,2.833801
2015-01-09,17.55,-0.110476,2.865054


In [3]:
# vix_training_start, vix_training_end = "2015-01-02", "2022-12-30"
vix_training_start, vix_training_end = "2018-01-01", "2020-06-30"
vix_testing_start, vix_testing_end = "2020-07-01", "2020-08-01"

vix_training_df = VIX[(VIX.index >= vix_training_start) & (VIX.index <= vix_training_end)]
vix_testing_df = VIX[(VIX.index >= vix_testing_start) & (VIX.index <= vix_testing_end)]
vix_training_df.head()

,vol,log_vol_diff,log_vol
2018-01-02,9.77,0.065563,2.279316
2018-01-03,9.15,-0.007621,2.213754
2018-01-04,9.22,0.000000,2.221375
2018-01-05,9.22,-0.032020,2.221375
2018-01-08,9.52,-0.057158,2.253395


In [4]:
from hurst import compute_Hc

vix_hurst_est, c, data = compute_Hc(vix_training_df["log_vol"], kind='change', simplified=True)
print(f"Estimated Hurst Exponent: {vix_hurst_est:.4f}")

Estimated Hurst Exponent: 0.6277


In [5]:
# Aggregated variance method

def hurst_aggregated_variance(series, min_window=10, max_window=None):
    """
    Estimate the Hurst exponent using Aggregated Variance method.
    
    Parameters:
    series : array-like
        The time series data.
    min_window : int
        Minimum window size.
    max_window : int or None
        Maximum window size (default is len(series) // 2).
    
    Returns:
    float
        Estimated Hurst exponent.
    """
    if max_window is None:
        max_window = len(series) // 2

    windows = np.logspace(np.log10(min_window), np.log10(max_window), num=20, dtype=int)
    var_values = []

    for w in windows:
        segments = [series[i:i+w] for i in range(0, len(series) - w, w)]
        segment_means = [np.mean(seg) for seg in segments]
        var_values.append(np.var(segment_means))

    log_var = np.log(var_values)
    log_n = np.log(windows[:len(log_var)])

    H, _ = np.polyfit(log_n, log_var, 1)  # Slope is 2H - 2
    return (H + 2) / 2

# Example Usage:
hurst_estimate_var = hurst_aggregated_variance(vix_training_df['log_vol'])
print(f"Estimated Hurst exponent (Aggregated Variance): {hurst_estimate_var}")

Estimated Hurst exponent (Aggregated Variance): 0.34803462555340936


In [ ]:
import pandas as pd
import numpy as np
from hurst import compute_Hc
import plotly.graph_objects as go

def compute_rolling_hurst(df, window, col='log_vol_diff', lag=0):
    hurst_values = []
    times = []
    
    # Loop through the DataFrame using the rolling window
    for i in range(window - 1, len(df), lag+1):
        # Extract the window slice from the series
        window_series = df[col].iloc[i - window + 1 : i + 1: lag+1]
        # Compute the Hurst exponent using the 'change' method and simplified calculation
        h, _, _ = compute_Hc(window_series, kind='change', simplified=True)
        
        hurst_values.append(h)
        times.append(df.index[i])
    
    # Create and return a new DataFrame with the computed Hurst exponents
    result_df = pd.DataFrame({'hurst': hurst_values}, index=times)
    return result_df

rolling_hurst_150 = compute_rolling_hurst(VIX, window=150, col='log_vol', lag=0)
rolling_hurst_350 = compute_rolling_hurst(VIX, window=300, col='log_vol', lag=0)
rolling_hurst_500 = compute_rolling_hurst(VIX, window=500, col='log_vol', lag=0)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=rolling_hurst_150.index,
    y=rolling_hurst_150["hurst"],
    mode='lines',
    name='150 days rolling window',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=rolling_hurst_350.index,
    y=rolling_hurst_350["hurst"],
    mode='lines',
    name='350 days rolling window',
    line=dict(color='red')
))
fig.add_trace(go.Scatter(
    x=rolling_hurst_500.index,
    y=rolling_hurst_500["hurst"],
    mode='lines',
    name='500 days rolling window',
    line=dict(color='green')
))
fig.update_layout(
    title=f"Hurst exponent for volatility",
    xaxis_title="Date",
    yaxis_title="Hurst Value",
    template="plotly_white"
)

### Test Simulations

In [ ]:
# Setup object
vix_training_data_obj = utils.VixDf(vix_training_df)

# Simulate 1 day
next_day_pred = utils.simulate_fbm_ndays(vix_training_data_obj, 0.3, 1)
print(f'Next day prediction: {next_day_pred}')

# Attempts to predict 5 days
utils.apply_rolling_function(vix_training_data_obj, 500, 5)
print(f'Next 5 days prediction: {next_day_pred}')

In [ ]:
vix_training_data_obj.get_df().head()

### Run fBM Model on data post-2022

In [ ]:
vix_obj = utils.VixDf(VIX)
result_df = utils.apply_rolling_predictions_from_start(vix_obj, '2022-01-01', 150)
result_df.head()

In [ ]:
merged_df = result_df.join(VIX, how='inner')
merged_df.head()

In [ ]:
utils.compute_rmse(merged_df, 'predicted', 'vol')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged_df.index,
    y=merged_df["predicted"],
    mode='lines',
    name='Predicted',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=merged_df.index,
    y=merged_df["vol"],
    mode='lines',
    name='Actual',
    line=dict(color='red')
))
fig.update_layout(
    title=f"Predicted vs Actual Volatility",
    xaxis_title="Date",
    yaxis_title="Volatility",
    template="plotly_white"
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats

# Example DataFrame: df should already have a time series index and the columns 'observed' and 'predicted'
# df = pd.read_csv('your_data.csv', index_col='date', parse_dates=True)

# 1. Compute the residuals
df = merged_df 
df['residual'] = df['vol'] - df['predicted']

# 2. Plot the residuals over time
plt.figure(figsize=(10, 4))
plt.plot(df.index, df['residual'], label='Residuals')
plt.title("Residuals Over Time")
plt.xlabel("Time")
plt.ylabel("Residual")
plt.legend()
plt.show()

Observations:
- Occasional large spikes, indicating outliers 
- Residuals oscillate around zero (good)
- No obvious trend

In [ ]:

# 3. Plot the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF)
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df['residual'], lags=20, ax=ax[0])
plot_pacf(df['residual'], lags=20, ax=ax[1])
ax[0].set_title("ACF of Residuals")
ax[1].set_title("PACF of Residuals")
plt.tight_layout()
plt.show()

Observations:
- No signification autocorrelation (since no spikes), captured temporal dependencies well
- Good fit

In [ ]:
# 4. Q-Q Plot for normality check of the residuals
plt.figure(figsize=(6, 6))
stats.probplot(df['residual'], dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals")
plt.show()

Observations:
- Observed heavy tails, indicating that there are outliers
- Non-normal distribution, (probably t distribution)

In [ ]:
# 5. Histogram of the residuals
plt.figure(figsize=(10, 4))
sns.histplot(df['residual'], kde=True)
plt.title("Histogram of Residuals")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.show()

Observations:
- roughly centered around zero, slightly negative, a bit of overpredicting

In [ ]:
# 6. Perform a normality test (Shapiro-Wilk Test)
stat, p_value = stats.shapiro(df['residual'])
print(f'Shapiro-Wilk Test statistic: {stat:.4f}, p-value: {p_value:.4f}')


Observations:
-  A very small p-value means we reject the null hypothesis that states that residuals are normally distributed.

Thoughts from plots:
- Standard fBM assumes increments follow a normal distribution, but clearly this isn't the case here.

### High Volatility Period

In [ ]:
vix_obj = utils.VixDf(VIX)
high_vol_df = utils.apply_rolling_predictions_from_start(vix_obj, '2020-01-01', 150)
high_vol_df = high_vol_df[(high_vol_df.index >= '2020-01-01') & (high_vol_df.index <= '2022-01-01')]
high_vol_df = high_vol_df.join(VIX, how='inner')
utils.compute_rmse(high_vol_df, 'predicted', 'vol')

### Low Vol Period

In [ ]:
vix_obj = utils.VixDf(VIX)
low_vol_df = utils.apply_rolling_predictions_from_start(vix_obj, '2018-01-01', 150)
low_vol_df = low_vol_df[(low_vol_df.index >= '2018-01-01') & (low_vol_df.index <= '2020-01-01')]
low_vol_df = low_vol_df.join(VIX, how='inner')
utils.compute_rmse(low_vol_df, 'predicted', 'vol')